In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from pathlib import Path
from scipy import spatial
path_to_dfs = (r'/home/abbas/Documents/2020_03_03_SJE/2019_05_03_fish_dfs/*')

In [2]:
dfs = glob.glob(path_to_dfs)

assert len(dfs) > 0, 'no files found'
#assert dfs[0].endswith('.csv')

In [3]:
len(dfs)


dfs_barbel = [df for df in dfs if 'barbel' in df]
dfs_grayling = [df for df in dfs if 'grayling' in df]
dfs_chub = [df for df in dfs if 'chub' in df]
print(len(dfs_barbel), len(dfs_grayling), len(dfs_chub))

134 100 6


In [4]:
def calc_max_gradient_direct(fish_flow_file,
                             flow_cat,
                             fish_nbr,
                             distance_thr=2):
    '''
    a function used to calculate between every position
    and closest 4 positions the difference in the
    depth gradient and flow velocity
    find the maximum difference and add it to the df

    https://stackoverflow.com/questions/53028514/
    calculate-distance-from-one-point-to-all-others
    '''
    fish_flow_df = pd.read_csv(fish_flow_file, sep=',', index_col=0,
                               parse_dates=True, engine='c')

#     fish_flow_df = fish_flow_df.head(10)

    flow_val = flow_cat[-2:]
    
    depth_var = 'depth_%s' % flow_val
    flow_var = 'velM_%s' % flow_val
    print(flow_val, depth_var, flow_var)
    #print('calculting for', fish_flow_file)

    x_grid_between = fish_flow_df.loc[:, 'X_of_grid_node']
    y_grid_between = fish_flow_df.loc[:, 'Y_of_grid_node']

    take_idxs = np.ones(x_grid_between.size, dtype=bool)
    for i in range(x_grid_between.size):

        if not take_idxs[i]:
            continue

        x_dists = (x_grid_between[i] - x_grid_between)**2
        y_dists = (y_grid_between[i] - y_grid_between)**2

        dists = (x_dists + y_dists)**0.5

        equal_dist_idxs = np.where((dists < 5e-1))[0]

        take_idxs[equal_dist_idxs[1:]] = False

    x_grid_between_unique_series = x_grid_between[take_idxs]
    y_grid_between_unique_series = y_grid_between[take_idxs]

    try:
        print('getting data')
        for ix in fish_flow_df.index:
            #print(ix)

            depth_point0 = fish_flow_df.loc[ix, depth_var]
            flow_vel_point0 = fish_flow_df.loc[ix, flow_var]

#             ix_before = ix - pd.Timedelta(minutes=timedelta)
#             ix_after = ix + pd.Timedelta(minutes=timedelta)

#             df_between = fish_flow_df.loc[ix_before:ix_after, :]

            #==================================================================
            x0 = fish_flow_df.loc[ix, 'X_of_grid_node']
            y0 = fish_flow_df.loc[ix, 'Y_of_grid_node']

            x_dists = (x0 - x_grid_between_unique_series.values)**2
            y_dists = (y0 - y_grid_between_unique_series.values)**2

            dists = (x_dists + y_dists)**0.5

            x_grid_between_unique_series_near = x_grid_between_unique_series[
                (0 < dists) & (dists <= distance_thr)]
            y_grid_between_unique_series_near = y_grid_between_unique_series[
                (0 < dists) & (dists <= distance_thr)]

            dists_below_thr = dists[(0 < dists) & (dists <= distance_thr)]

            if dists_below_thr.size > 0:

                nrsts_idxs = np.argsort(dists_below_thr)[:5]
                x_nebors = x_grid_between_unique_series_near[nrsts_idxs]
                y_nebors = y_grid_between_unique_series_near[nrsts_idxs]

                #==============================================================
#                 if (x0 in x_nebors) and (y0 in y_nebors):
#                     x_nebors = list(x_nebors.values)
#                     y_nebors = list(y_nebors.values)
#                     i = 0  # loop counter
#                     length = len(x_nebors)  # list length
#                     while (i < length):
#                         if (x_nebors[i] == x0) and (y_nebors[i] == y0):
#                             print('removing x0, y0 from points')
#                             x_nebors.remove(x_nebors[i])
#                             y_nebors.remove(y_nebors[i])
#                             # as an element is removed
#                             # so decrease the length by 1
#                             length = length - 1
#                             # run loop again to check element
#                             # at same index, when item removed
#                             # next item will shift to the left
#                             continue
#                         i = i + 1

                x_nebors_arr_noX0 = np.array(x_nebors)
                y_nebors_arr_noY0 = np.array(y_nebors)

#                 plt.ioff()
#                 plt.scatter(x_nebors, y_nebors, c='g', alpha=0.75)
#                 plt.scatter(x_fish, y_fish, c='orange', alpha=0.5)
#                 plt.scatter(x0, y0, c='r', alpha=0.5)

                diff_in_grds_lst, diff_in_vel_lst = [], []
                for xnear, ynear in zip(x_nebors_arr_noX0, y_nebors_arr_noY0):
                    if xnear == x0 and ynear == y0:
                        print('Error Same grid point')
                        raise Exception

                    df_xynear = fish_flow_df[
                        (fish_flow_df.loc[:, 'X_of_grid_node'] == xnear) &
                        (fish_flow_df.loc[:, 'Y_of_grid_node'] == ynear)]

                    point_i_depth = df_xynear.loc[:, depth_var].values
                    point_i_flow_mag = df_xynear.loc[:, flow_var].values

                    diff_in_grds_lst.append(
                        np.abs(depth_point0 - point_i_depth))
                    diff_in_vel_lst.append(
                        np.abs(flow_vel_point0 - point_i_flow_mag))

                diff_in_grds_lst_unique = [
                    np.unique(arr) for arr in diff_in_grds_lst]
                diff_in_vel_lst_unique = [
                    np.unique(arr) for arr in diff_in_vel_lst]

                (_, depth_point_id) = (
                    np.max(diff_in_grds_lst_unique),
                    np.argmax(diff_in_grds_lst_unique))

                (_, vel_point_id) = (
                    np.max(diff_in_vel_lst_unique),
                    np.argmax(diff_in_vel_lst_unique))

                (x_d, y_d) = (x_nebors[depth_point_id],
                              y_nebors[depth_point_id])
                (x_v, y_v) = (x_nebors[vel_point_id],
                              y_nebors[vel_point_id])
                assert not (x_d == x0 and y_d ==
                            y0), 'Error gradient same point'
                assert not (x_v == x0 and y_v ==
                            y0), 'Error gradient same point'

#                 plt.scatter(x_d, y_d, c='darkblue')
#                 plt.scatter(x_v, y_v, c='orange')

                direction_max_depth_grd = np.math.degrees(
                    np.math.atan2((y_d - y0), (x_d - x0)))
                direction_max_vel_grd = np.math.degrees(
                    np.math.atan2((y_v - y0), (x_v - x0)))

#                 fish_flow_df.loc[ix,
#                                  'X_coords_grid_with_max_%s_gradient_diff'
#                                  % depth_var] = x_d
#
#                 fish_flow_df.loc[ix,
#                                  'Y_coords_grid_with_max_%s_gradient_diff'
#                                  % depth_var] = y_d
#
#                 fish_flow_df.loc[ix,
#                                  'X_coords_grid_with_max_%s_gradient_diff'
#                                  % flow_var] = x_v
#
#                 fish_flow_df.loc[ix,
#                                  'Y_coords_grid_with_max_%s_gradient_diff'
#                                  % flow_var] = y_v

                fish_flow_df.loc[ix,
                                 'Direction_max_%s_gradient_compared_to_grid_point_and_x_axis'
                                 % depth_var] = direction_max_depth_grd
                fish_flow_df.loc[ix,
                                 'Direction_max_%s_gradient_compared_to_grid_point_and_x_axis'
                                 % flow_var] = direction_max_vel_grd

                diff_fish_max_depth_grd = np.mod(
                    # 'fish_angle'
                    fish_flow_df.loc[ix, 'Fish_swim_direction_compared_to_x_axis'] -
                    direction_max_depth_grd + 180, 360) - 180

                diff_fish_max_vel_grd = np.mod(
                    fish_flow_df.loc[ix, 'Fish_swim_direction_compared_to_x_axis'] -
                    direction_max_vel_grd + 180, 360) - 180

                fish_flow_df.loc[ix,
                                 'Angle_between_swim_and_max_%s_gradient_direction'
                                 % depth_var] = diff_fish_max_depth_grd
                fish_flow_df.loc[ix,
                                 'Angle_between_swim_and_max_%s_gradient_direction'
                                 % flow_var] = diff_fish_max_vel_grd
            else:
                #print('Assigning Nans because no nearest neighbour found')

#                 fish_flow_df.loc[ix,
#                                  'X_coords_grid_with_max_%s_gradient_diff'
#                                  % depth_var] = np.nan
#
#                 fish_flow_df.loc[ix,
#                                  'Y_coords_grid_with_max_%s_gradient_diff'
#                                  % depth_var] = np.nan
#
#                 fish_flow_df.loc[ix,
#                                  'X_coords_grid_with_max_%s_gradient_diff'
#                                  % flow_var] = np.nan
#
#                 fish_flow_df.loc[ix,
#                                  'Y_coords_grid_with_max_%s_gradient_diff'
#                                  % flow_var] = np.nan

                fish_flow_df.loc[ix,
                                 'Direction_max_%s_gradient_compared_to_grid_point_and_x_axis'
                                 % depth_var] = np.nan
                fish_flow_df.loc[ix,
                                 'Direction_max_%s_gradient_compared_to_grid_point_and_x_axis'
                                 % flow_var] = np.nan

                fish_flow_df.loc[ix,
                                 'Angle_between_swim_and_max_%s_gradient_direction'
                                 % depth_var] = np.nan
                fish_flow_df.loc[ix,
                                 'Angle_between_swim_and_max_%s_gradient_direction'
                                 % flow_var] = np.nan
    except Exception as msg:
        print(msg)
    print('saving df')
#     fish_flow_df.rename(
#         columns={'Velocity': 'Fish_swim_velocity_m_per_s',
#                  'x_fish': 'Fish_x_coord',
#                  'y_fish': 'Fish_y_coord',
#                  'index_of_grid_node': 'Index_of_grid_node',
#                  'fish_angle': 'Fish_swim_direction_compared_to_x_axis',
#                  'flow_angle': 'Flow_direction_compared_to_x_axis',
#                  'angle_diff': 'Angle_between_swim_and_flow_direction'},
#         inplace=True)
#     deltax = fish_flow_df.Fish_x_coord.diff()
#     deltay = fish_flow_df.Fish_x_coord.diff()
#     fish_flow_df['Time'] = fish_flow_df.index
#     fish_flow_df['Time_difference_in_s'] = np.round(
#         fish_flow_df.Time.diff() / pd.Timedelta('1s'), 1)
#     fish_flow_df['Traveled_distance_in_m'] = calculate_distance_2_points(deltax,
#                                                                          deltay)
#         fish_flow_df_final.drop('Time', axis=1, inplace=True)

    cols_new = ['Longitude', 'Latitude', 'Fish_x_coord',
                'Fish_y_coord', 'Time_difference_in_s',
                'Traveled_distance_in_m',
                'Fish_swim_velocity_m_per_s', 'HPE', 'RMSE',
                'Flow_Cat', 'Index_of_grid_node',
                'X_of_grid_node', 'Y_of_grid_node', 'Z_of_grid_node',
                'depth_%s' % flow_val, 'velX_%s' % flow_val,
                'velY_%s' % flow_val, 'velM_%s' % flow_val,
                'Fish_swim_direction_compared_to_x_axis',
                'Flow_direction_compared_to_x_axis',
                'Angle_between_swim_and_flow_direction',
                #                 'X_coords_grid_with_max_%s_gradient_diff'
                #                 % depth_var,
                #                 'Y_coords_grid_with_max_%s_gradient_diff'
                #                 % depth_var,
                #                 'X_coords_grid_with_max_%s_gradient_diff'
                #                 % flow_var,
                #                 'Y_coords_grid_with_max_%s_gradient_diff'
                #                 % flow_var,
                'Direction_max_%s_gradient_compared_to_grid_point_and_x_axis'
                % depth_var,
                'Direction_max_%s_gradient_compared_to_grid_point_and_x_axis'
                % flow_var,
                'Angle_between_swim_and_max_%s_gradient_direction'
                % depth_var,
                'Angle_between_swim_and_max_%s_gradient_direction'
                % flow_var,
                'group']#,
                #'Depth_Grad']
    # change column names and save df
    fish_flow_df_final = fish_flow_df[cols_new]

    fish_flow_df_final.to_csv(
        os.path.join(r'/run/media/abbas/EL Hachem 2019/from_C/Desktop/Work_with_Matthias_Schneider/2020_05_03_data/',
                     r'fish_%s_with_flow_data_%s_angles'
                     r'_and_max_gradients_behavior_new.csv'
                     % (fish_nbr, flow_cat)), float_format='%0.6f')  # , compression='gzip')
    print('Done saving df')
    return fish_flow_df_final


In [8]:
alls = len(dfs_barbel)

bad_dfs = []
for df in dfs_barbel:
    
    if 'barbel' in df:
        fish_number = df.split('/')[-1][5:19]
        flow_cat = df.split('/')[-1][39:41]
    if 'grayling' in df:
        fish_number = df.split('/')[-1][5:21]
        flow_cat = df.split('/')[-1][37:43]
    if 'chub' in df:
        fish_number = df.split('/')[-1][5:17]
        flow_cat = df.split('/')[-1][33:39]
        
    print(alls, '++', fish_number, ' * ', flow_cat)
    
    try:
        calc_max_gradient_direct(df, flow_cat, fish_number)
        
    except Exception as msg:
        print(msg)
        bad_dfs.append(df)
        continue
        #raise Exception
    print('DONE WITH THIS ONE')
        
    alls -= 1
    

134 ++ 2_barbel_46838  *  10
10 depth_10 velM_10
getting data


<ipython-input-4-8fcf030a8f8e>:181: RuntimeWarning: invalid value encountered in remainder
  diff_fish_max_depth_grd = np.mod(
<ipython-input-4-8fcf030a8f8e>:186: RuntimeWarning: invalid value encountered in remainder
  diff_fish_max_vel_grd = np.mod(


saving df
Done saving df
DONE WITH THIS ONE
133 ++ 2_barbel_46849  *  30
30 depth_30 velM_30
getting data
saving df
Done saving df
DONE WITH THIS ONE
132 ++ 2_barbel_46860  *  40
40 depth_40 velM_40
getting data
saving df
Done saving df
DONE WITH THIS ONE
131 ++ 2_barbel_46844  *  10
10 depth_10 velM_10
getting data
saving df
Done saving df
DONE WITH THIS ONE
130 ++ 2_barbel_46850  *  10
10 depth_10 velM_10
getting data
saving df
Done saving df
DONE WITH THIS ONE
129 ++ 2_barbel_46860  *  20
20 depth_20 velM_20
getting data
saving df
Done saving df
DONE WITH THIS ONE
128 ++ 2_barbel_46846  *  30
30 depth_30 velM_30
getting data
saving df
Done saving df
DONE WITH THIS ONE
127 ++ 2_barbel_46861  *  30
30 depth_30 velM_30
getting data
saving df
Done saving df
DONE WITH THIS ONE
126 ++ 2_barbel_46856  *  60
60 depth_60 velM_60
getting data
saving df
Done saving df
DONE WITH THIS ONE
125 ++ 2_barbel_46856  *  20
20 depth_20 velM_20
getting data
saving df
Done saving df
DONE WITH THIS ONE
12

KeyboardInterrupt: 

In [280]:
df

'/home/abbas/Documents/2020_03_03_SJE/2019_05_03_fish_dfs/desktop.ini'

In [271]:

for df in df2[:1]:
    print(df)
    del (df)
    

/home/abbas/Documents/2020_03_03_SJE/2019_05_03_fish_dfs/fish_2_barbel_46838_with_flow_data_cat_10_angles_and_max_gradients_with_behavior.csv


In [261]:
fish_number = df.split('/')[-1][5:19]
flow_cat = df.split('/')[-1][35:41]

In [263]:
flow_cat
fish_number

'2_barbel_46838'